<a href="https://colab.research.google.com/github/null-buster/PDF_Summarizer/blob/main/T5_Abstractive_Summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install transformers
!pip install torch
!pip install sentencepiece
!pip install pdfminer.six

Importing required packages

In [18]:
import torch
import json 
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
#import pdfminer
import os
import io
import io
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage
#print(pdfminer.__version__)

In [19]:
def pre_process_text(text):
  preprocess_text = text.strip().replace("\n","")
  t5_prepared_Text = "summarize: "+preprocess_text
  #print ("original text preprocessed: \n", preprocess_text)
  return preprocess_text,t5_prepared_Text

In [20]:
def get_summary(t5_prepared_Text):
  tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)
  
  # summmarize 
  summary_ids = model.generate(tokenized_text,
                                      num_beams=4,
                                      no_repeat_ngram_size=2,
                                      min_length=30,
                                      max_length=100,
                                      early_stopping=True)
  output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
  return output
  

In [21]:
#using the t5-base pretrained checkpoint trained on 220 million parameters
model = T5ForConditionalGeneration.from_pretrained('t5-base')
tokenizer = T5Tokenizer.from_pretrained('t5-base')
device = torch.device('cpu')


Some weights of the model checkpoint at t5-base were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [22]:
def extract_text_by_page(pdf_path):
    with open(pdf_path, 'rb') as fh:
        for page in PDFPage.get_pages(fh, 
                                      caching=True,
                                      check_extractable=True):
            resource_manager = PDFResourceManager()
            fake_file_handle = io.StringIO()
            converter = TextConverter(resource_manager, fake_file_handle)
            page_interpreter = PDFPageInterpreter(resource_manager, converter)
            page_interpreter.process_page(page)
            
            text = fake_file_handle.getvalue()
            yield text
    
            # close open handles
            converter.close()
            fake_file_handle.close()


In [23]:
def extract_text(pdf_path):
  final_summary = ""
  page_count = 1
  for page in extract_text_by_page(pdf_path):
    #print(page)
    #print(type(page))
    text = page
    preprocess_text, t5_prepared_Text = pre_process_text(text)
    #print ("original text preprocessed: \n", preprocess_text)
    output = get_summary(t5_prepared_Text)
    #print ("\n\nSummarized text: \n",output)
    print()
    final_summary += output
    #print(final_summary)
    final_summary += "\n\n"
    print("Done Summarizing Page----------->>> " + str(page_count))
    page_count += 1
  print("\n\nSummarization done. Printing summary.\n")
  return final_summary


In [24]:
def read_pdf_paths(folder_name, output_folder):
  path_to_files = os.listdir(os.path.abspath('.') + '/' + folder_name)
  for entry in path_to_files:
    if entry.split('.')[-1] == 'pdf':
      print("Processing Pdf : " + entry)
      final_summary = extract_text(os.path.abspath('.') + '/' + folder_name + '/' + entry)
      with open(os.path.abspath('.') + '/' + output_folder + '/Summary_' + entry.split('.')[-2] + '.txt', 'w' ) as f:
        f.write("Summary of " + entry.split('.')[-2]+":\n\n" )
        f.write("\n\n===========================================\n\n" )
        f.write(final_summary)
      #print(entry)
      print("Summary of " + entry.split('.')[-2]+" is :\n\n")
      print(final_summary)



### Output: 
The code below created the input and outputs folders for the model to take inputs from and then reads each pdf page by page to extract summaries and then saves the summaries as plain text files.

Steps for running the script:

1. Run all the cells above this point.


2.   Run the cell below to create the input folder containing the Pdfs with the name `pdf_folder` and the output folder that'll contain the output summaries as text files with the name `summary_folder`.

Note : This step could be skipped if these folders are already present or if you wish to manually create these folders.

3.   Populate the `pdf_folder` with the Pdfs to be summarized.



In [25]:
pdf_folder = '/pdf_folder'
if not os.path.exists(os.path.abspath('.') + pdf_folder):
  os.mkdir(os.path.abspath('.') + pdf_folder)
summary_folder = '/summary_folder'
if not os.path.exists(os.path.abspath('.') + summary_folder):
  os.mkdir(os.path.abspath('.') + summary_folder)



3.   Run the code below with the appropriate folder names created in the previous step. The program will read all the Pdfs present in the `pdf_folder` and output the results in the `summary_folder`.


In [27]:
read_pdf_paths('pdf_folder', 'summary_folder')

Processing Pdf : The-COVID-19-recovery-will-be-digital-A-plan-for-the-first-90-days-vF_677b26fd2626e24c8ab1bf28ed70410e.pdf

Done Summarizing Page----------->>> 1

Done Summarizing Page----------->>> 2

Done Summarizing Page----------->>> 3

Done Summarizing Page----------->>> 4

Done Summarizing Page----------->>> 5

Done Summarizing Page----------->>> 6

Done Summarizing Page----------->>> 7

Done Summarizing Page----------->>> 8


Summarization done. Printing summary.

Summary of The-COVID-19-recovery-will-be-digital-A-plan-for-the-first-90-days-vF_677b26fd2626e24c8ab1bf28ed70410e is :


COVID-19 recovery will be digital: A plan for the first 90 days. rapid migration to digital technologies driven by the pandemic will continue into the recovery.

digital will undoubtably play a center-stage role during the recovery and beyond. banks have transitioned to remote sales and service teams, schools have pivoted to online learning and digital classrooms, doctors have begun delivering telem